# Quesiti e Tasks Statistica e Machine Learning

-> [Scaica il Database](https://octospa-my.sharepoint.com/personal/a_sauro_octotelematics_com/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fa%5Fsauro%5Foctotelematics%5Fcom%2FDocuments%2Fyellow%5Ftripdata%5F2019%2D04%2Ecsv%2Ezip&parent=%2Fpersonal%2Fa%5Fsauro%5Foctotelematics%5Fcom%2FDocuments&ga=1)

**INDICE:**
1. Database "tripdata"
2. Quesito A
3. Quesito A1
4. Quesito B
___

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
df_tripdata = pd.read_csv(r"yellow_tripdata_2019-04.csv")

## 1. Database "tripdata"
___
In questa fase iniziale, mi occupo della preparazione del database per le analisi successive. Per prima cosa, verifico il numero di colonne e righe presenti. 

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) 

In [4]:
df_tripdata.shape

(7433139, 18)

In [5]:
df_tripdata.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge'], dtype='object')

Controllo la presenza dei valori nulli ed elimino le colonne che non servono all'analisi.

In [6]:
valori_null = df_tripdata.isnull().sum()
null_columns = valori_null[valori_null > 0]

if null_columns.empty:
    print("NON SONO PRESENTI VALORI NULLI")
else:
    print(f"COLONNE CON VALORI NULLI: \n{null_columns}")

NON SONO PRESENTI VALORI NULLI


In [7]:
df_tripdata.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID',
                          'DOLocationID', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge'],inplace=True)

In [8]:
df_tripdata.columns

Index(['VendorID', 'passenger_count', 'trip_distance', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount'], dtype='object')

In [9]:
df_tripdata.shape

(7433139, 7)

## 2. Quesito A

Calcolare sull’intero dataset i percentili 5, 50 e 95 (q05, q50, q95) sui valori del dataset: 
**'fare_amount'**,**'tip_amount'** e **'total_amount'** ; suddivisi secondo i campi **'VendorID'**,**'passenger_count'** e **'payment_type'**.

L’output di calcolo dovrà essere un dataFrame da esportare in formato CSV 
organizzato con: 

Colonne: nome campo (sul quale è calcolato il percentile) + “_p_” + soglia percentile.

Righe (index): nome campo di reggruppamento + ”_” + valore del gruppo sul quale è eseguito il calcolo dei percentili.
___

Definisco i campi su cui calcolare i percentili, poi con la funzione calcolo i percentili.

In [10]:
fields = ['fare_amount', 'tip_amount', 'total_amount']
group_fields = ['VendorID', 'passenger_count', 'payment_type']
percentiles = [5, 50, 95]

In [11]:
def calculate_percentiles(df_tripdata, fields, group_fields, percentiles):
    results = []

    for group_field in group_fields:
        grouped = df_tripdata.groupby(group_field)
        for group, data in grouped:
            result_row = {'group': f"{group_field}_{group}"}
            for field in fields:
                for percentile in percentiles:
                    col_name = f"{field}_p_{percentile}"
                    result_row[col_name] = data[field].quantile(percentile / 100)
            results.append(result_row)
    return pd.DataFrame(results)

percentile_df = calculate_percentiles(df_tripdata, fields, group_fields, percentiles)

## 3. Quesito A1

Calcolare i percentili come riportati per il quesito A anche per il dataset suddiviso per **'trip_distance'** se >2.8 o <=2.8 e aggiungere i valori calcolati al dataFrame con le logiche riportare al quesito A
___

Suddivido per la colonna **'trip_distance'** raggruppati in:
- maggiori di 2.8,
- minori o uguali a 2.8.

Calcolo i percentili, poi aggiungo l'output al dataframe.

In [12]:
def calculate_distance_percentiles(df_tripdata, fields, percentiles):
    results = []

    df_tripdata['distance_group'] = df_tripdata['trip_distance'].apply(lambda x: '>2.8' if x > 2.8 else '<=2.8')
    grouped = df_tripdata.groupby('distance_group')

    for group, data in grouped:
        result_row = {'group': f"trip_distance_{group}"}
        for field in fields:
            for percentile in percentiles:
                col_name = f"{field}_p_{percentile}"
                result_row[col_name] = data[field].quantile(percentile / 100)
        results.append(result_row)

    return pd.DataFrame(results)

trip_distance_percentile_df = calculate_distance_percentiles(df_tripdata, fields, percentiles)

percentile_df = pd.concat([percentile_df, trip_distance_percentile_df], ignore_index=True)

Creo una pabella pivot per visualizzare l'output.

In [13]:
pivot_table = pd.pivot_table(
    percentile_df,
    index='group',
    aggfunc='first')

pivot_table

,fare_amount_p_5,fare_amount_p_50,fare_amount_p_95,tip_amount_p_5,tip_amount_p_50,tip_amount_p_95,total_amount_p_5,total_amount_p_50,total_amount_p_95
group,,,,,,,,,
VendorID_1,4.500,9.5,36.000,0.0,1.86,6.5500,8.300,14.75,48.300
VendorID_2,4.500,9.5,40.000,0.0,1.96,7.3600,8.300,14.76,52.700
VendorID_4,4.000,9.0,35.275,0.0,1.96,7.1175,8.300,14.75,48.650
passenger_count_0,4.000,9.0,37.500,0.0,1.85,6.6500,8.300,14.72,49.800
passenger_count_1,4.500,9.5,37.000,0.0,1.95,7.0000,8.300,14.75,49.900
passenger_count_2,4.500,9.5,42.000,0.0,1.86,7.2000,8.300,14.80,54.500
passenger_count_3,4.500,9.5,42.000,0.0,1.85,7.0000,8.300,14.80,54.537
passenger_count_4,4.500,10.0,50.000,0.0,1.58,6.9900,8.750,15.30,58.920
passenger_count_5,4.500,9.5,37.500,0.0,1.96,7.1800,8.300,14.76,50.520


Esportazione del dataframe in csv

In [14]:
pivot_table.to_csv('report.csv')

## 4. Quesito B

Generare un modello di ML per la stima del “total_amount” in funzione delle variabili (in input al modello): **'VendorID'**, **'passenger_count'**, **'payment_type'**, **'trip_distance'**.

È possibile definire autonomamente la metodologia e il processo di selezione e split del dataset di riferimento per allenamento, test e verifica del modello (k-f, random, train-test-valid).

*Per l’ottimizzazione del modello si consiglia di calcolare il RMSE sul parziale dataset di test selezionato.*
___

Seleziono delle variabili indipendenti e dipendenti.

In [15]:
X = df_tripdata[['VendorID', 'passenger_count', 'payment_type', 'trip_distance']]
y = df_tripdata['total_amount']

Codifico le variabili categoriche se necessario.

In [16]:
X = pd.get_dummies(X, columns=['VendorID', 'payment_type'], drop_first=True)

Divido il dataset in training, validation e test set:
- Training set (60%): Usato per allenare il modello.
- Validation set (20%): Usato per valutare e ottimizzare il modello durante lo sviluppo.
- Test set (20%): Usato per una valutazione finale e imparziale del modello.

In [17]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Inizializzazione e allenamento del modello.

In [18]:
model = LinearRegression()
model.fit(X_train, y_train)

print("Coefficients:", model.coef_)

Coefficients: [ 8.15007244e-03  3.30158923e+00 -1.55716015e-01  1.61995372e-01
 -3.44121199e+00 -6.78108901e+00 -1.20991707e+01]


 Valutzione dell'importanza relativa delle feature. 
 
 *Coefficienti più alti in valore assoluto indicano maggiore impatto sulla predizione.*

In [19]:
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_})

print(f"Feature Importance: \n{feature_importance}")

Feature Importance: 
           Feature  Coefficient
0  passenger_count     0.008150
1    trip_distance     3.301589
2       VendorID_2    -0.155716
3       VendorID_4     0.161995
4   payment_type_2    -3.441212
5   payment_type_3    -6.781089
6   payment_type_4   -12.099171


RMSE (Root Mean Square Error) sul test set

In [20]:
y_test_pred = model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {rmse_test}")

Test RMSE: 10.1766235932388


Esportazione del modello

In [21]:
with open('ml.pkl', 'wb') as f: pickle.dump(model, f)

Esportazione del codice in file *.txt*

In [ ]:
import nbformat

with open("Progetto Octo.ipynb", "r", encoding="utf-8") as notebook_file:
    notebook_content = nbformat.read(notebook_file, as_version=4)

notebook_text = []
for cell in notebook_content['cells']:
    if cell['cell_type'] == 'markdown':
        notebook_text.append(f"# {cell['source']}\n")
    elif cell['cell_type'] == 'code':
        notebook_text.append("python\n")
        notebook_text.append(cell['source'])
        notebook_text.append("\n\n")

with open("code.txt", "w", encoding="utf-8") as text_file:
    text_file.write("\n".join(notebook_text))